# Example of using HW-DMD for traffic speed forecasting
This notebook shows a minimal, reproducible example of using [HW-DMD](https://github.com/mcgill-smart-transport/high-order-weighted-DMD) [1] for short-term traffic speed forecasting on the [Seattle Inductive Loop Detector Dataset](https://github.com/zhiyongc/Seattle-Loop-Data). The original dataset contains every five-minute traffic speed recorded by 323 loop detectors in Seattle for the year 2015. Here we use the first month's data for demonstration. The locations of the detectors are shown in the following figure.

<img src="DataLoop.png" title="DataLoop" width="500"/>

We use the traffic speed of the last ten intervals $[\mathbf{v}_{t-1}, \mathbf{v}_{t-2}, \cdots, \mathbf{v}_{t-10}]$ to forecast the traffic speed $\mathbf{v}_{t}$ in the next time interval. This setting is the same as the [original paper](https://github.com/zhiyongc/Graph_Convolutional_LSTM) [2] that used this Seattle traffic dataset. We will see HW-DMD is comparable with many deep-learning models in the [original paper](https://github.com/zhiyongc/Graph_Convolutional_LSTM).

References:
- [1] Cheng, Z., Trepanier, M., & Sun, L. (2021). Real-time forecasting of metro origin-destination matrices with high-order weighted dynamic mode decomposition. arXiv preprint arXiv:2101.00466.
- [2] Cui, Z., Henrickson, K., Ke, R., & Wang, Y. (2019). Traffic graph convolutional recurrent neural network: A deep learning framework for network-scale traffic learning and forecasting. IEEE Transactions on Intelligent Transportation Systems, 21(11), 4883-4894.

## Define HW-DMD and helper functions

In [1]:
import numpy as np
from scipy.linalg import svd, orth
from numpy.linalg import pinv, eigh
import time

class HWDMD:
    def __init__(self, h, rx, ry, rho=1, bs=1):
        self.h = np.sort(h)
        self.rx = rx
        self.ry = ry
        self.rho = rho # Forgetting ratio, i.e. weights
        self.sigma = rho ** 0.5
        self.bs = bs  # batch size
        self.Ux = None
        self.Uy = None
        self.P = None
        self.Qx = None
        self.Qy = None
        self.Qx_inv = None
        self.n = None

    def Uxi(self, i):
        if i == len(self.h):  # The Ux for other features
            return self.Ux[(i * self.n):, :]
        else:  # The Ux for auto-regression
            return self.Ux[(i * self.n):(i * self.n + self.n), :]

    def fit(self, X, Y):
        self.n, t = Y.shape
        m = Y.shape[1]

        num_bs = np.ceil(t / self.bs)  # Number of batches
        weight = self.sigma ** np.repeat(np.arange(num_bs), self.bs)[:m][::-1]

        X = X * weight
        Y = Y * weight

        [Ux, _, _] = svd(X, full_matrices=False)
        [Uy, _, _] = svd(Y, full_matrices=False)
        self.Ux = Ux[:, 0:self.rx]
        self.Uy = Uy[:, 0:self.ry]

        Xtilde = self.Ux.T @ X
        Ytilde = self.Uy.T @ Y
        self.P = Ytilde @ Xtilde.T
        self.Qx = Xtilde @ Xtilde.T
        self.Qy = Ytilde @ Ytilde.T
        self.Qx_inv = pinv(self.Qx)

    def forecast(self, X):
        """Return the one_step forecast for **staggered** data."""
        nn = len(self.h)
        part2 = 0
        for i in range(nn):
            part2 += self.Uxi(i).T @ self.Uy @ (self.Uy.T @ X[i * self.n:(i * self.n + self.n), :])

        # When there are external features
        if X.shape[0] > nn*self.n:
            part2 += self.Uxi(nn).T @ X[nn * self.n:, :]

        return self.Uy @ self.P @ self.Qx_inv @ part2

    def update_model(self, X, Y):
        """Update the model coefficients using the new data (staggered). Does not change the buffer data"""
        if X.shape[1] != self.bs or Y.shape[1] != self.bs:
            raise ValueError('Number of columns does not equal to batchsize.')
        Ybar = self.Uy @ (self.Uy.T @ Y)
        Xbar = self.Ux @ (self.Ux.T @ X)

        Ux = orth(X - Xbar)
        Uy = orth(Y - Ybar)
        self.Ux = np.concatenate([self.Ux, Ux], axis=1)
        self.Uy = np.concatenate([self.Uy, Uy], axis=1)
        self.Qx = np.pad(self.Qx, ((0, Ux.shape[1]), (0, Ux.shape[1])), 'constant', constant_values=0)
        self.Qy = np.pad(self.Qy, ((0, Uy.shape[1]), (0, Uy.shape[1])), 'constant', constant_values=0)
        self.P = np.pad(self.P, ((0, Uy.shape[1]), (0, Ux.shape[1])), 'constant', constant_values=0)

        Ytilde = self.Uy.T @ Y
        Xtilde = self.Ux.T @ X
        self.P = self.rho * self.P + Ytilde @ Xtilde.T
        self.Qx = self.rho * self.Qx + Xtilde @ Xtilde.T
        self.Qy = self.rho * self.Qy + Ytilde @ Ytilde.T

        # Compress rx and ry
        evalue_x, evector_x = eigh(self.Qx)
        evalue_y, evector_y = eigh(self.Qy)
        evector_x = evector_x[:, -self.rx:]
        evector_y = evector_y[:, -self.ry:]
        self.Ux = self.Ux @ evector_x
        self.Uy = self.Uy @ evector_y
        self.Qx = np.diag(evalue_x[-self.rx:])
        self.Qy = np.diag(evalue_y[-self.ry:])
        self.P = evector_y.T @ self.P @ evector_x

        # Update
        self.Qx_inv = pinv(self.Qx)


# Some helper functions
def stagger_data(data, h):
    # Help to prepare input and target
    h.sort()
    len_h = len(h)
    n, m = data.shape
    max_h = max(h)

    Y = data[:, max_h:]
    X = np.zeros((n * len_h, m - max_h), dtype=data.dtype)
    for i in range(len_h):
        X[i * n: i * n + n, :] = data[:, max_h - h[i]:m - h[i]]
    return X, Y


def RMSE(f0, f1):
    return np.sqrt(np.mean((f0 - f1) ** 2))


def MAPE(f0, f1):
    return np.mean(np.abs(f0-f1)/f0) * 100

## Prepare data

In [2]:
import pandas as pd
data = pd.read_pickle('..//data//Seattle_one_month_speed_2015.pickle')

train_idx = np.arange(0, 25*24*12)  # Use the first 25 days for training
test_idx = np.arange(25*24*12, 31*24*12)  # The next 6 days for testing
h = np.arange(1,11)
X, Y = stagger_data(data.iloc[train_idx, :].values.T, h)
X_test, Y_test = stagger_data(data.iloc[test_idx[0]-len(h):, :].values.T, h)

## V1: a constant HW-DMD without online update

In [3]:
# The optimal `rx` is usually larger than `ry`
model = HWDMD(h, rx=300, ry=90, rho=0.999, bs=12)
t0 = time.time()
model.fit(X, Y)
Y_predict = model.forecast(X_test)
print(f'Total time: {time.time()-t0:.3f}s')
print(f'RMSE:{RMSE(Y_test, Y_predict):.3f}')
print(f'MAPE:{MAPE(Y_test, Y_predict):.3f}%')

Total time: 43.956s
RMSE:4.579
MAPE:8.172%


## V2: online update HW-DMD every one hour

In [4]:
bs = 12  # The batch size in the online update
Y_predict2 = []
t0 = time.time()
for i in range(144):
    X_today = X_test[:, i*bs:(i+1)*bs]
    Y_today = Y_test[:, i*bs:(i+1)*bs]
    Y_predict2.append(model.forecast(X_today))
    model.update_model(X_today, Y_today)
print(f'Total time: {time.time()-t0:.3f}s')
Y_predict2 = np.concatenate(Y_predict2, axis=1)
print(f'RMSE:{RMSE(Y_test, Y_predict2):.3f}')
print(f'MAPE:{MAPE(Y_test, Y_predict2):.3f}%')

Total time: 4.624s
RMSE:4.504
MAPE:7.955%


Note: RMSE=4.63, MAPE=6.01% for the best model in the [original paper](https://github.com/zhiyongc/Graph_Convolutional_LSTM) [2] that used this Seattle traffic dataset. They probably used the entirely year data? This notebook uses one-month data.

## Summary
- Implementing HW-DMD from raw is simple (around one hundred lines).
- The estimation is fast.
- The performance is comparable to some complex deep-learning models.
- Properly use the online update can improve forecast accuracy. 